<a href="https://colab.research.google.com/github/alhichri/Heartprint/blob/main/load_data_and_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code load ECG data, preprocesses it, and visulaizes it

# New Section

In [ ]:
import tensorflow as tf
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import glob
import scipy.io

from sklearn.metrics import confusion_matrix,  classification_report, accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.utils import shuffle
import matplotlib.image as mpimg
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

tf.__version__


'2.8.0'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)

Mounted at /content/gdrive


In [ ]:
###################################################################################################
########################## Parameters:  ##########################
dataset_path = '/content/gdrive/MyDrive/000research/13-INF2168-02-R/Data-Master/Session-1/ECG/Heartbeats/'

# dataset_folder = 'original_data/'; image_format = '.png'
# images_folder = dataset_path + dataset_folder + 'images_folder/'; print(images_folder)

# dataset_folder = 'original_data/'; image_format = '.png'
# images_folder = dataset_path + dataset_folder + 'images_folder/'; print(images_folder)

dataset_folder = 'Session-1'; image_format = '.gz'
images_folder = dataset_path + dataset_folder + '/'; print(images_folder)

List_of_folders = os.listdir(images_folder)

print('List of folders are: ', List_of_folders)
for x in List_of_folders:
    print(x)

num_of_classes  = len(List_of_folders)
print('num_of_classes: ' , num_of_classes)

print(images_folder+  List_of_folders[0])
F_idx = []; list_of_files= []
for cn in range(num_of_classes):
    data_dir = pathlib.Path(images_folder+  List_of_folders[cn] + '/')
    flist = list(data_dir.glob('*'+image_format)) 
    list_of_files.append(  flist   )

print(list_of_files)

In [ ]:
mat = scipy.io.loadmat(dataset_path+'sig_s1.mat')
# X = mat['X']; y = mat['y']; NUM_CLASSES = int(y.max()+1); print(NUM_CLASSES)
print(mat.keys())
h = mat['sig']
print(h.shape)
number_of_subjects = h.shape[1]
for s in range(number_of_subjects):
  h0 = h[0 , s]
  print( s , h0.shape  )

In [ ]:
dataset_path = '/content/gdrive/MyDrive/000research/13-INF2168-02-R/Data-Master/Session-1/ECG/Heartbeats/'

mat = scipy.io.loadmat(dataset_path+'hb_p9_s1.mat')
# X = mat['X']; y = mat['y']; NUM_CLASSES = int(y.max()+1); print(NUM_CLASSES)
print(mat.keys())
h = mat['hbs']
print(h.shape)
number_of_subjects = h.shape[1]

# we have a problem with subject 145  h[0,145]  has shape 0,0
X = h[0 , s] ; Y = np.zeros( ( X.shape[0],1)  ); 
label= 1
for s in range(1,number_of_subjects):
  h0 = h[0 , s]
  # print( 'heartbeats: ' , h0.shape , ' label: ', s)
  if h0.shape[0]>0:
    X = np.vstack((X , h0))
    labels = label * np.ones( ( h0.shape[0],1));     Y = np.vstack((Y ,  labels   ))
    label = label + 1

Y = Y.astype('int')
print( X.shape , Y.shape)

dict_keys(['__header__', '__version__', '__globals__', 'hbs'])
(1, 168)
(5653, 227) (5653, 1)


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X, Y));     #del Y_pool, Y_pool  # to save RAM
# dataset_generated.concatenate( train_ds )
# print(' Two datasets are merged.')

In [ ]:
def configure_for_performance(ds , batch_size=8, dropRemainder=True , reshuffle_each_iteration=True ):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size=1000 , reshuffle_each_iteration=reshuffle_each_iteration)
  ds = ds.batch(batch_size , drop_remainder=dropRemainder)
  ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

train_ds_batch = configure_for_performance(train_ds , batch_size=8, dropRemainder=True )